## Reading & Lesson:

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\steve\Documents\CF - Instacart Project\02 Data'

In [4]:
ords_prods_merge=pd.read_pickle(os.path.join(path,'Prepared Data', 'orders_products_updated.pkl'))

In [5]:
# Creating a table with only the first 1,000,000 lines
df = ords_prods_merge[:1000000]

In [6]:
df.shape

(1000000, 16)

In [7]:
# Grouping content by product_name
df.groupby('product_name')

In [15]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders


### Performing a Single Aggregation:

In [8]:
# df.groupby('category') .agg({'category': ['aggregation']})
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [29]:
# Alternate way to get the mean
df.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

In [31]:
# Even better way to write it out
df.groupby('department_id').order_number.mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

While the .'s are more natural, its probably harder for others to tell what you are doing and also results in my work on Python since .'s are used for multiple things while []'s are only used for indexing.

Due to that, it may be better to stick with []'s

### To Perform Multiple Aggregations:

In [32]:
# More aggregations are simply listed in the brackets
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

### Transforming Data:

In [9]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

In [26]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty flag,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders,10,NaN,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,NaN,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders,10,NaN,New customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,NaN,New customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most orders,10,NaN,New customer


### Deriving Columns w/ loc():

In [21]:
# To remove the max row limit, so you can view all the rows in a request:
pd.options.display.max_rows = None

In [22]:
# Using loc() to find values > 40 in max_order column and label Loyal customer in new loyalty_flag column
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty flag'] = 'Loyal customer'

In [23]:
ords_prods_merge.loc[(ords_prods_merge['max_order']<=40) & (ords_prods_merge['max_order']>10), 'loyalty_flag'] = 'Regular customer'

In [24]:
ords_prods_merge.loc[ords_prods_merge['max_order']<=10, 'loyalty_flag'] = 'New customer'

In [25]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
NaN                 10284093
New customer         6243990
Name: count, dtype: int64

In [27]:
# Checking head but specifying columns
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10
